In [2]:
# build a database from a dataset for nomenclating
# INPUT: a collection of GAIN domain PDBs, their sequences as one large ".fa" file
from gain_classes import GainDomain, GainCollection, Anchors, GPS
import sse_func
import execute
import numpy as np
import glob
import multiprocessing as mp
from subprocess import Popen, PIPE


In [3]:
# The Goal is to create a GainCollection from all GAIN domains here
# The set of proteins needs to be filtered and analyzed statistically before

# Example PDB name:
#  R0LVI9-R0LVI9_ANAPL-PutativeGR125-Anas_platyrhynchos_unrelaxed_rank_1_model_3.pdb
# corresponding STRIDE name:
#  R0LVI9-R0LVI9_ANAPL-PutativeGR125-Anas_platyrhynchos.stride
#pdbs = glob.glob("/home/hildilab/projects/agpcr_nom/*/batch*/*_rank_1_*.pdb")
pdbs = glob.glob("/home/hildilab/projects/agpcr_nom/all_gps_celsr/batch*/*_rank_1_*.pdb")
print(f"Found {len(pdbs)} best ranked models in target directories.")
#print(len(celsr_pdbs))

Found 0 best ranked models in target directories.


In [3]:
stride_folder = f"/home/hildilab/projects/agpcr_nom/all_gps_stride"
stride_bin = "/home/hildilab/lib/stride/stride"
           
def compile_stride_mp_list(pdbs, stride_folder,stride_bin):
    stride_mp_list = []
    
    for pdb in pdbs:
        pdb_name = pdb.split("/")[-1]
        name = pdb_name.split("_unrelaxed_")[0]
        out_file = f"{stride_folder}/{name}.stride"
        arg = [pdb, out_file, stride_bin]
        
        stride_mp_list.append(arg)
        
    return stride_mp_list

def run_stride(arg):
    pdb_file, out_file, stride_bin = arg
    stride_command = f"{stride_bin} {pdb_file} -f{out_file}"
    execute.run_command(stride_command)

def execute_stride_mp(stride_mp_list, n_threads=10):
        stride_pool = mp.Pool(n_threads)
        stride_pool.map(run_stride, stride_mp_list)
        print("Completed mutithreaded creation of STRIDE files!")
        
        #execute.run_stride(pdb, out_file, stride_bin)

In [4]:
#stride_mp_list = compile_stride_mp_list(celsr_pdbs, stride_folder, stride_bin)
#stride_mp_list = compile_stride_mp_list(singlet_pdbs, stride_folder, stride_bin)
#print(len(stride_mp_list))
#[print(arg) for arg in stride_mp_list[:10]]
# MP execution of STRIDE
#execute_stride_mp(stride_mp_list, n_threads=2)

In [6]:
# Eliminate double entries (both in the original run and the added small runs)
# Form the "pdbs" list

stride_files = glob.glob("/home/hildilab/projects/agpcr_nom/all_gps_stride/*")
accessions = [f.split(".strid")[0].split("/")[-1] for f in stride_files]
pdb_accessions = np.array([p.split("_unrelaxed_")[0].split("/")[-1] for p in pdbs])

# Find duplicate in the original pdbs list and indicate them via > is_duplicate = True <
is_duplicate=np.zeros([len(pdbs)], dtype=bool)
sort_pdb_ac = np.sort(pdb_accessions)
duplicate_list = []
for i, pdb in enumerate(sort_pdb_ac):
    if i+1 == len(sort_pdb_ac):
        break
    if pdb == sort_pdb_ac[i+1]:
        duplicate_list.append(pdb)
        multi_indices = np.where(pdb == pdb_accessions)[0]
        is_duplicate[multi_indices[0]] = True

np_pdbs = np.array(pdbs)
singlet_pdbs = np_pdbs[is_duplicate == False] # This is the reduced list with ONLY UNIQUE PDBs
print(f"Reduced the initial set of {len(pdbs)} PDB files down to {len(singlet_pdbs)} files.")


# This is a check routine if there are PDBs in the reduced list which have NOT a STRIDE file
singlet_pdb_accessions = np.array([p.split("_unrelaxed_")[0].split("/")[-1] for p in singlet_pdbs])

counter = 0
for ac in singlet_pdb_accessions:
    if ac not in accessions:
        print(ac)
    else:
        counter += 1
print(f"Found {len(singlet_pdb_accessions)-counter}/{len(singlet_pdb_accessions)} accessions in the accession list.")

Reduced the initial set of 1373 PDB files down to 1373 files.
Found 0/1373 accessions in the accession list.


In [7]:
"""stride_folder = f"{pdb_folder}/stride_files"
fasta_file = "/home/hildilab/projects/agpcr_nom/batch_files/batch_60.fa"
# CHECK FILES!
import os
if not os.path.isfile(fasta_file) or os.path.islink(fasta_file):
    print("ERROR: Specify FASTA FILE CONTAINING ALL SEQUENCES OF THE PROTEINS!")
    
    
alignment_file = "/home/hildilab/projects/agpcr_nom/batch_out_test/mafft.fa"
#alignment_file = None

# If there is not a specified base alignment, create one (that might take a while tho.)
if not alignment_file:
    mafft_bin = "mafft"
    mafft_command = f"{mafft_bin} --auto --thread 4 {fasta_file}"
    out_dir = "/".join(fasta_file.split("/")[:-2])
    out_file = f"{out_dir}/mafft.fa"
    execute.run_command(mafft_command, out_file = out_file)
    alignment_file = out_file
gps_minus_one_column = 1209"""

'stride_folder = f"{pdb_folder}/stride_files"\nfasta_file = "/home/hildilab/projects/agpcr_nom/batch_files/batch_60.fa"\n# CHECK FILES!\nimport os\nif not os.path.isfile(fasta_file) or os.path.islink(fasta_file):\n    print("ERROR: Specify FASTA FILE CONTAINING ALL SEQUENCES OF THE PROTEINS!")\n    \n    \nalignment_file = "/home/hildilab/projects/agpcr_nom/batch_out_test/mafft.fa"\n#alignment_file = None\n\n# If there is not a specified base alignment, create one (that might take a while tho.)\nif not alignment_file:\n    mafft_bin = "mafft"\n    mafft_command = f"{mafft_bin} --auto --thread 4 {fasta_file}"\n    out_dir = "/".join(fasta_file.split("/")[:-2])\n    out_file = f"{out_dir}/mafft.fa"\n    execute.run_command(mafft_command, out_file = out_file)\n    alignment_file = out_file\ngps_minus_one_column = 1209'

In [4]:
# The GainCollection class needs to be modified to also parse a list of sequences 
# instead of a folder containing one fasta per seq
class FilterCollection:
    ''' 
    A collection of GainDomain objects used for filtering a set of AF2 models
    This is used to condense the dataset towards one containing only GAIN domains

    Attributes
    ----------

    collection : list
        List of GainDomain instances
   
    valid_gps : np.array(bool)
        For each protein, specify if the GPS detection is valid or not
    
    valid_subdomain : np.array(bool)
        For each protein, specify if it has detected subdomains or not
    
    Methods
    ----------
    print_gps(self):
        Prints info about all GainDomain GPS subinstances

    write_all_seq(self, savename):
        Writes all sequences in the Collection to one file

    transform_alignment(self, input_alignment, output_alignment, aln_cutoff):
        Transforms a given alignment with SSE data to a SSE-assigned version of the alignment
        
    write_filtered(self, savename, bool_mask):
        Writes all sequences to File where a boolean mask (i.e. subdomain_criterion, gps_criterion)
        is True at respective position
    '''
    def __init__(self,
                alignment_file, 
                aln_cutoff,
                quality,
                gps_index,
                stride_files,
                sequence_files=None, # modified to object containing all seqs
                sequences=None, # replaces sequence_files
                subdomain_bracket_size=20,
                domain_threshold=20,
                coil_weight=0,
                alignment_dict=None): 
        '''
        Constructs the GainCollection objects by initializing one GainDomain instance per given sequence
        
        Parameters
        ----------
        alignment_file:     str, required
            The base dataset alignment file

        aln_cutoff:         int, required
            Integer value of last alignment column

        quality:            list, required
            List of quality valus for each alignment column. Has to have $aln_cutoff items
            By default, would take in the annotated Blosum62 values from the alignment exported from Jalview

        gps_index:          int, required
            Alignment column index of the GPS-1 residue (consensus: Leu)
        
        stride_files:       list, required
            A list of stride files corresponding to the sequences contained within.
        
        sequence_files:     list, optional
            A list of sequence files to be read as the collection - specify either this
            or sequences as an object instead of files for sequences
        
        sequences:          object, optional
            A list of (sequence_name, sequence) tuples containing all sequences. Can be specified
            instead of sequence_files

        subdomain_bracket_size: int, optional
            Smoothing window size for the signal convolve function. Default = 20.

        domain_threshold:   int, optional
            Minimum size of a helical segment to be considered candidate for Subdomain A of GAIN domain. Default = 20.

        coil_weight:        float, optional
            Weight assigned to unordered residues during Subdomain detection. Enables decay of helical signal
            default = 0. Recommended values < +0.2 for decay

        Returns
        ----------
        None
        '''
        # Initialize collection (containing all GainDomain instances) and the anchor-histogram
        if sequence_files:
            # Compile all sequence files to a sequences object
            sequences = np.empty([len(sequence_files)])
            for i, seq_file in enumerate(sequence_files):
                name, seq = sse_func.read_seq(seq_file, return_name=True)
                sequences[i] = (name, seq)
        elif (sequences is not None):
            print(f"Found sequences object.")
        else: 
            print(f"ERROR: no sequence_files or sequences parameter found. Aborting compilation.")
            return None
        self.collection = np.empty([len(sequences)], dtype=object)
        
        valid_gps = np.zeros([len(sequences)], dtype=bool)
        valid_subdomain = np.zeros([len(sequences)], dtype=bool)
        #anchor_hist = np.zeros([aln_cutoff])#
        # Create a GainDomain instance for each sequence file contained in the list
        for i,seq_tup in enumerate(sequences):
            # updated GainDomain class
            name, sequence = seq_tup
            explicit_stride = [stride for stride in stride_files if name.split("-")[0] in stride]
            if len(explicit_stride) == 0:
                print(f"Stride file not found for {name}")
                continue
            newGain = GainDomain(alignment_file, 
                                  aln_cutoff,
                                  quality,
                                  name = name,
                                  sequence = sequence,
                                  gps_index = gps_index, 
                                  subdomain_bracket_size = subdomain_bracket_size,
                                  domain_threshold = domain_threshold,
                                  coil_weight = coil_weight,
                                  explicit_stride_file = explicit_stride[0],
                                  without_anchors = True,
                                  skip_naming = True,
                                  alignment_dict = alignment_dict)

            # Check if the object staisfies minimum criteria
            if newGain.isValid: 
                  
                self.collection[i] = newGain
                  
                if newGain.hasSubdomain:
                    valid_subdomain[i] = True
                if newGain.GPS.isConsensus:
                    valid_gps[i] = True
        
        self.valid_subdomain = valid_subdomain
        self.valid_gps = valid_gps
        print(f"Completed Collection initialitazion of {len(sequences)} sequences.\n"
             f"{np.count_nonzero(self.collection)} valid proteins were found.\n"
             f"{np.count_nonzero(self.valid_subdomain)} of which have detected Subdomains.\n"
             f"{np.count_nonzero(self.valid_gps)} of which have detected consensus GPS motifs.\n")

    def print_gps(self):
        '''
        Prints information about the GPS of each GAIN domain.

        Parameters:
            None
        Returns:
            None
        '''
        for i, gain in enumerate(self.collection):
            try:
                gain.GPS.info()
            except:
                print(f"No GPS data for {gain.name}. Likely not a GAIN Domain!")

    def write_all_seq(self, savename):
        '''
        Write all GAIN sequences of the GainCollection into one fasta file.

        Parameters
        ----------
        savename: str, required
            Output name of the fasta file.

        Returns
        ----------
        None
        '''
        with open(savename, 'w') as f:
            for gain in self.collection:
                f.write(f">{gain.name[:-3]}\n{''.join(gain.sequence)}\n")
    
    def write_filtered(self, savename, bool_mask=None, write_mode='w'):
                  
        '''
        Internal function for writing filtered sequences to file.
        Takes the Gain.sequence np.array type to write the truncated versions.
        
        Parameters
        ----------
        savename: str, required
            Output name of the fasta file.
        bool_mask: list/array, required
            A mask of len(self.collection) where a boolean defines whether to write the
            sequence to file or not
        Returns
        ----------
        None
        '''
        with open(savename, write_mode) as f:
            print(f"writing filtered to {savename}")
            for i, gain in enumerate(self.collection):
                if gain is not None and bool_mask[i] == True:
                    f.write(f">{gain.name.replace('.fa','')}\n{''.join(gain.sequence)}\n")

    def transform_alignment(self, input_alignment, output_alignment, aln_cutoff):
        ''' 
        Transform any input alignment containing all sequences in the GainCollection 
        into one where each residue is replaced with the respective 
        Secondary Structure from the STRIDE files

        Parameters
        ----------
        input_alignment: str, required
            Input alignment file
        output_alignment: str, required
            Output alignment file
        aln_cutoff: int, required
            Last alignment column to be read from the Input Alignment

        Returns
        ---------
        None
        '''
        initial_dict = sse_func.read_alignment(input_alignment, aln_cutoff)
        out_dict = {}
        for gain in self.collection:
            sse_alignment_row = np.full([aln_cutoff], fill_value='-', dtype='<U1')
            mapper = sse_func.get_indices(gain.name, gain.sequence, input_alignment, aln_cutoff)
            for index, resid in enumerate(gain.sse_sequence):
                sse_alignment_row[mapper[index]] = resid
            out_dict[gain.name[:-3]] = sse_alignment_row

        # Write to file
        with open(output_alignment, "w") as f:
            for key in out_dict.keys():
                f.write(f">{key}\n{''.join(out_dict[key])}\n")
        print(f"Done transforming alignment {input_alignment} to {output_alignment} with SSE data.")
            

In [9]:
#alignment_file = "/home/hildilab/projects/agpcr_nom/batch_out_test/big_mafft.fa"
#quality_file = "/home/hildilab/projects/agpcr_nom/batch_out_test/big_mafft.jal"
alignment_file = "/home/hildilab/projects/GPS_massif/uniprot_query/trunc_celsr.mafft.fa"
quality_file = "/home/hildilab/projects/GPS_massif/uniprot_query/trunc_celsr.mafft.jal"
fasta_file = "/home/hildilab/projects/GPS_massif/uniprot_query/all_celsr_trunc.fa"
#stride_folder = f"{pdb_folder}/stride_files"
quality = sse_func.read_quality(quality_file)
gps_minus_one = 4966 #19258
aln_cutoff = 4990 #19822
sequences = sse_func.read_multi_seq(fasta_file)
print(len(sequences))
stride_files = glob.glob("/home/hildilab/projects/agpcr_nom/all_gps_stride/*")
print(len(stride_files))

1745
15954


In [10]:
"""filterCollection = FilterCollection(alignment_file,
                                   aln_cutoff = 19822,
                                   quality = quality,
                                   gps_index = gps_minus_one,
                                   stride_files = stride_files,
                                   sequences = sequences)"""

'filterCollection = FilterCollection(alignment_file,\n                                   aln_cutoff = 19822,\n                                   quality = quality,\n                                   gps_index = gps_minus_one,\n                                   stride_files = stride_files,\n                                   sequences = sequences)'

In [11]:
#filterCollection.write_filtered(savename="test.fa", bool_mask = filterCollection.valid_gps, write_mode='w')

In [5]:
def batch_filter_seqs(arg_item):
                   # [sequences,      # A number of sequences as tuple instances
                   #  stride_folder,  # A folder containing ALL stride files
                   #   output_prefix,  # A prefix for individual file identification
                   #   alignment_file, # The big (initial) alignment file
                   #   quality,        # The corr. parsed quality for BLOSUM62 score
                   #   aln_cutoff,     # the left-most column (19822 for big_mafft.fa)
                   #   gps_minus_one,  # The column index of GPS-1 (zero-indexed! 19258 big_mafft)
                   #   ]
    sequences, stride_folder, output_prefix, alignment_file, quality, aln_cutoff, gps_minus_one, alignment_dict = arg_item
    # Parallelizable version of filtering sequences and models via FilterCollection
    # This should create separate files for each valid, fragment and no-SD group
    # These files should then be grouped together
    # The batch size is arbitrary and is considered the number of sequences passed
    
    # Output: Profiles; 4 Text files (valid, fragment, invalidGPS, invalid)
    filteredBatch = FilterCollection(alignment_file,
                                   aln_cutoff = aln_cutoff,
                                   quality = quality,
                                   gps_index = gps_minus_one,
                                   stride_files = stride_files,
                                   sequences = sequences,
                                   alignment_dict = alignment_dict)
    outpath = "/home/hildilab/projects/agpcr_nom/all_gps_profiles"
    
    for Gain in filteredBatch.collection:
        if Gain:
            Gain.plot_profile(outdir=outpath)
        
    suffixes = ["gain", 
                "fragments", 
                "noncons_gps", 
                "invalid"]
    masks = [np.logical_and(filteredBatch.valid_gps, filteredBatch.valid_subdomain),
              np.logical_not(filteredBatch.valid_subdomain),
              np.logical_not(filteredBatch.valid_gps),
              np.logical_not(np.logical_and(filteredBatch.valid_gps, filteredBatch.valid_subdomain))]
    # write four separate files, matching each criterion
    for k in range(4):
        filteredBatch.write_filtered(savename=f"{outpath}/{output_prefix}_{suffixes[k]}.fa", 
                                     bool_mask = masks[k],
                                     write_mode = 'w')
    del filteredBatch
    return None 

In [6]:
def run_mp_collection(arg_list, n_threads=10):
    pool = mp.Pool(n_threads)
    pool.map(batch_filter_seqs, arg_list)
    print("Completed mutithreaded filtering.")

def construct_arg_list(batch_sequence_files, 
                       output_folder,
                       stride_folder, 
                       quality, 
                       alignment_file, 
                       aln_cutoff, 
                       gps_minus_one,
                       alignment_dict = None):
    """ each item looks like this:
        sequences, \ 
        stride_folder, \
        output_prefix, \
        alignment_file, \
        quality, \
        aln_cutoff, \
        gps_minus_one = arg_item"""
    # static : stride_folder, quality, alignment_file, aln_cutoff, gps_minus_one
    # flexible : sequences, output_prefix
    arg_list = []
    #
    for idx, sequence_file in enumerate(batch_sequence_files):
        
        index_string = str(idx)
        sequences = sse_func.read_multi_seq(sequence_file)
        output_prefix = f"{output_folder}_{index_string.zfill(3)}"
        item = [sequences, 
                stride_folder, 
                output_prefix, 
                alignment_file, 
                quality, 
                aln_cutoff, 
                gps_minus_one,
                alignment_dict]
        
        arg_list.append(item)
    
    print(f"[NOTE] : Compiled list of arguments for multithreaded filtering"
          f" containing {len(arg_list)} items.")
    return arg_list

In [9]:
# 
batch_sequence_files = glob.glob("/home/hildilab/projects/agpcr_nom/all_gp*/batch_*.fa")
print(len(batch_sequence_files))
output_folder = "app_gain_domains"

alignment_file = "/home/hildilab/projects/agpcr_nom/appended_big_mafft.fa" # This is a combined alignment of ALL sequences in ALL queries!
quality_file = "/home/hildilab/projects/agpcr_nom/appended_big_mafft.jal"  # ^ corresponding quality file
stride_folder = "/home/hildilab/projects/agpcr_nom/all_gps_stride" 
quality = sse_func.read_quality(quality_file)
gps_minus_one = 21160  # 19258
aln_cutoff = 21813 # 19822
stride_files = glob.glob("/home/hildilab/projects/agpcr_nom/all_gps_stride/*")
alignment_dict = sse_func.read_alignment(alignment_file, aln_cutoff)
print(len(stride_files))
print(len(batch_sequence_files))
#print(quality)
#sequences = sse_func.read_multi_seq(fasta_file)

arg_list = construct_arg_list(batch_sequence_files, 
                       output_folder,
                       stride_folder, 
                       quality, 
                       alignment_file, 
                       aln_cutoff, 
                       gps_minus_one,
                       alignment_dict)


465
15954
465
[NOTE] : Compiled list of arguments for multithreaded filtering containing 465 items.


In [15]:
for k in range(480, len(arg_list)+1):
    batch_filter_seqs(arg_list[k])
#run_mp_collection(arg_list[510:], n_threads=24)
#for arg in arg_list[-5:]: print(arg[0])

Found sequences object.
Completed Collection initialitazion of 75 sequences.
75 valid proteins were found.
75 of which have detected Subdomains.
75 of which have detected consensus GPS motifs.

writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_480_gain.fa
writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_480_fragments.fa
writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_480_noncons_gps.fa
writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_480_invalid.fa
Found sequences object.
No Subdomain boundaries detected. Possible Fragment found.
No GPS residue present in this Domain. Is this really a GAIN?
[DEBUG] gain_classes.GPS : ALTERNATIVE GPS DETECTION! DEBUG:last_sheets = [(260, 266), (273, 282)]
No Subdomain boundaries detected. Possible Fragment found.
No GPS residue present in this Domain. Is this really a GAIN?
[DEBUG] gain_classes.GPS :

[WARNING] sse_func.find_boundaries : There are still secondary-structure associated residues within the Subdomain connecting loop. Please check if this is within limits:
scored_seq[helix_end+1:sheet_start] = array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0, -1, -1,  0,  0,  0,  0])
[WARNING] sse_func.find_boundaries : There are still secondary-structure associated residues within the Subdomain connecting loop. Please check if this is within limits:
scored_seq[helix_end+1:sheet_start] = array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0, -1, -1,  0,  0,  0,  0])
[WARNING] sse_func.find_boundaries : There are still secondary-structure associated residues within the Subdomain connecting loop. Please check if this is within limits:
scored_seq[helix_end+1:sheet_start] = array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0, 

writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_495_gain.fa
writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_495_fragments.fa
writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_495_noncons_gps.fa
writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_495_invalid.fa
Found sequences object.
No Subdomain boundaries detected. Possible Fragment found.
No GPS residue present in this Domain. Is this really a GAIN?
[DEBUG] gain_classes.GPS : ALTERNATIVE GPS DETECTION! DEBUG:last_sheets = [(702, 704), (709, 711)]
No GPS residue present in this Domain. Is this really a GAIN?
[DEBUG] gain_classes.GPS : ALTERNATIVE GPS DETECTION! DEBUG:last_sheets = [(731, 734), (742, 745)]
Completed Collection initialitazion of 75 sequences.
75 valid proteins were found.
74 of which have detected Subdomains.
73 of which have detected consensus GPS motifs.

writing fi

writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_506_gain.fa
writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_506_fragments.fa
writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_506_noncons_gps.fa
writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_506_invalid.fa
Found sequences object.
Completed Collection initialitazion of 75 sequences.
75 valid proteins were found.
75 of which have detected Subdomains.
75 of which have detected consensus GPS motifs.

writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_507_gain.fa
writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_507_fragments.fa
writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_507_noncons_gps.fa
writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain

This is not a GAIN domain.
No Subdomain boundaries detected. Possible Fragment found.
This is not a GAIN domain.
No Subdomain boundaries detected. Possible Fragment found.
This is not a GAIN domain.
No Subdomain boundaries detected. Possible Fragment found.
No GPS residue present in this Domain. Is this really a GAIN?
[DEBUG] gain_classes.GPS : ALTERNATIVE GPS DETECTION! DEBUG:last_sheets = [(86, 95), (100, 104)]
This is not a GAIN domain.
No Subdomain boundaries detected. Possible Fragment found.
This is not a GAIN domain.
No Subdomain boundaries detected. Possible Fragment found.
No Strands detected. This is not a GAIN domain.
This is not a GAIN domain.
No Subdomain boundaries detected. Possible Fragment found.
Completed Collection initialitazion of 75 sequences.
63 valid proteins were found.
4 of which have detected Subdomains.
40 of which have detected consensus GPS motifs.

writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_508_gain.fa
writing 

Completed Collection initialitazion of 75 sequences.
75 valid proteins were found.
75 of which have detected Subdomains.
75 of which have detected consensus GPS motifs.

writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_518_gain.fa
writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_518_fragments.fa
writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_518_noncons_gps.fa
writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_518_invalid.fa
Found sequences object.
Completed Collection initialitazion of 75 sequences.
75 valid proteins were found.
75 of which have detected Subdomains.
75 of which have detected consensus GPS motifs.

writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_519_gain.fa
writing filtered to /home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_519_fragments.fa
writing filtered

IndexError: list index out of range

In [17]:

def compile_fastas(prefix, out_prefix):
    # Compiles the fasta files together to construct one large file containing the sequences
    # satisfying each criterion in the 2x2 matrix
    # we want to have the GAIN sequence only that is output by the write_filtered() func.
    
    # Gather all files:
    suffixes = ["gain", 
                "fragments", 
                "noncons", 
                "invalid"]
    all_files = np.asarray(glob.glob(f"{prefix}*fa"))
    print(len(all_files))
    for suffix in suffixes:
        sub_list = sorted([f for f in all_files if suffix in f.split("_")[-1]])
        print(f"Sublist constructed for {suffix = } containing {len(sub_list)} files.")
        with open(f"{out_prefix}_{suffix}.fa", "w") as all_file:
            all_seqs = []
            for file in sub_list:
                seqs = sse_func.read_multi_seq(file)
                for j in seqs:
                    if j in all_seqs:
                        print(j[0], "doublet")
                        continue
                    all_seqs.append(j)
                    all_file.write(f">{j[0]}\n{j[1]}\n")

compile_fastas("/home/hildilab/projects/agpcr_nom/all_gps_profiles/app_gain_domains_",
              out_prefix = "/home/hildilab/projects/agpcr_nom/app_gain")

2096
Sublist constructed for suffix = 'gain' containing 524 files.
A0A7K9SLG0-A0A7K9SLG0_9AVES-CELR1protein-Galbula_dea. doublet
A0A7K9T1D2-A0A7K9T1D2_9AVES-CELR3protein-Galbula_dea. doublet
A0A7K9TLI9-A0A7K9TLI9_9AVES-CELR1protein-Chloroceryle_aenea doublet
A0A7K9TVP7-A0A7K9TVP7_9AVES-CELR3protein-Chloroceryle_aenea doublet
A0A7K9VHH2-A0A7K9VHH2_ANSSE-CELR1protein-Anseranas_semipalmata doublet
A0A7K9VWE0-A0A7K9VWE0_9PASS-CELR3protein-Rhipidura_dahli. doublet
A0A7K9XS61-A0A7K9XS61_9GRUI-CELR3protein-Psophia_crepitans doublet
A0A7K9XX47-A0A7K9XX47_9GRUI-CELR1protein-Psophia_crepitans doublet
A0A7L0BJY9-A0A7L0BJY9_9AVES-CELR1protein-Spizaetus_tyrannus doublet
A0A7L0BKU1-A0A7L0BKU1_9AVES-CELR3protein-Spizaetus_tyrannus doublet
A0A6J2ACI7-A0A6J2ACI7_ACIJB-cadherinEGFLAGseven-passG-typereceptor1-Acinonyx_jubatus doublet
A0A1V4JZT1-A0A1V4JZT1_PATFA-CadherinEGFLAGseven-passG-typereceptor1isoformC-Patagioenas_fasciata_monilis. doublet
A0A1V4JZT5-A0A1V4JZT5_PATFA-CadherinEGFLAGseven-passG-typer

A0A3Q0DKT9-A0A3Q0DKT9_CARSF-cadherinEGFLAGseven-passG-typereceptor1-Carlito_syrichta doublet
A0A3Q0G3C0-A0A3Q0G3C0_ALLSI-cadherinEGFLAGseven-passG-typereceptor3-Alligator_sinensis doublet
A0A2I0MHB3-A0A2I0MHB3_COLLI-Cadherin,EGFLAGseven-passG-typereceptor1-Columba_livia doublet
A0A0Q3MB40-A0A0Q3MB40_AMAAE-Cadherin,EGFLAGseven-passG-typereceptor3-Amazona_aestiva doublet
A0A7K8J5Z8-A0A7K8J5Z8_9PASS-CELR3protein-Chaetorhynchus_papuensis doublet
A0A7K8JJA2-A0A7K8JJA2_9GRUI-CELR1protein-Lophotis_ruficrista. doublet
A0A7K8L315-A0A7K8L315_9GRUI-CELR3protein-Ardeotis_kori. doublet
A0A7K8L6I2-A0A7K8L6I2_9GRUI-CELR1protein-Ardeotis_kori. doublet
A0A7K8QAW1-A0A7K8QAW1_9PASS-CELR3protein-Smithornis_capensis. doublet
A0A7K9AVR5-A0A7K9AVR5_DRONO-CELR1protein-Dromaius_novaehollandiae doublet
A0A7K6IFW8-A0A7K6IFW8_9PASS-CELR1protein-Dasyornis_broadbenti doublet
A0A7K6JBJ4-A0A7K6JBJ4_9CORV-CELR3protein-Machaerirhynchus_nigripectus. doublet
A0A7K6JT20-A0A7K6JT20_9PASE-CELR3protein-Oreocharis_arfaki doub

A0A6J0IPK6-A0A6J0IPK6_9PASS-cadherinEGFLAGseven-passG-typereceptor3-Lepidothrix_coronata doublet
A0A6J0TVT0-A0A6J0TVT0_9SAUR-cadherinEGFLAGseven-passG-typereceptor1isoformX2-Pogona_vitticeps doublet
A0A6J0TWD4-A0A6J0TWD4_9SAUR-cadherinEGFLAGseven-passG-typereceptor1isoformX3-Pogona_vitticeps doublet
A0A6J0U057-A0A6J0U057_9SAUR-cadherinEGFLAGseven-passG-typereceptor1isoformX1-Pogona_vitticeps doublet
A0A6J0U351-A0A6J0U351_9SAUR-cadherinEGFLAGseven-passG-typereceptor3-Pogona_vitticeps doublet
A0A6P8Q5N5-A0A6P8Q5N5_GEOSA-cadherinEGFLAGseven-passG-typereceptor3-Geotrypetes_seraphini doublet
A0A850X012-A0A850X012_PIACA-CELR1protein-Piaya_cayana doublet
A0A850YBY7-A0A850YBY7_AEGCA-CELR3protein-Aegithalos_caudatus doublet
U3J9Z3-U3J9Z3_ANAPP-CadherinEGFLAGseven-passG-typereceptor1-Anas_platyrhynchos_platyrhynchos doublet
U3K608-U3K608_FICAL-CadherinEGFLAGseven-passG-typereceptor1-Ficedula_albicollis doublet
A0A2Y9QAI9-A0A2Y9QAI9_DELLE-cadherinEGFLAGseven-passG-typereceptor1isoformX2-Delphinap

A0A5A9PKU5-A0A5A9PKU5_9TELE-AGRE1-Triplophysa_tibetana. doublet
A0A7F8QVN6-A0A7F8QVN6_LEPWE-AGRB2isoformX1-Leptonychotes_weddellii doublet
A0A7F8QWZ6-A0A7F8QWZ6_LEPWE-AGRB2isoformX3-Leptonychotes_weddellii doublet
A0A7K5W6D3-A0A7K5W6D3_9SYLV-AGRA2protein-Hylia_prasina doublet
A0A7K7X325-A0A7K7X325_9PASS-AGRD2protein-Mohoua_ochrocephala. doublet
A0A7K9DJJ5-A0A7K9DJJ5_9AVES-AGRD2protein-Hemiprocne_comata. doublet
F6TY05-F6TY05_MACMU-AGRE5-Macaca_mulatta doublet
A0A2Y9ERN5-A0A2Y9ERN5_PHYMC-AGRG2isoformX2-Physeter_macrocephalus doublet
A0A452ITM8-A0A452ITM8_9SAUR-AGRA3-Gopherus_agassizii doublet
A0A4W2II23-A0A4W2II23_BOBOX-AGRG2-Bos_indicus_x_Bos_taurus doublet
A0A4X2JPY3-A0A4X2JPY3_VOMUR-AGRG2-Vombatus_ursinus doublet
A0A5F5PFL6-A0A5F5PFL6_HORSE-AGRG2-Equus_caballus doublet
A0A5F5PG86-A0A5F5PG86_HORSE-AGRG2-Equus_caballus doublet
A0A5F8GJT4-A0A5F8GJT4_MONDO-AGRF3-Monodelphis_domestica doublet
A0A5F8GLI1-A0A5F8GLI1_MONDO-AGRF3-Monodelphis_domestica doublet
A0A5G2QPV6-A0A5G2QPV6_PIG-AGRG2-S

A0A7K5CX97-A0A7K5CX97_9TYRA-AGRA3protein-Pachyramphus_minor. doublet
A0A1S3J0B6-A0A1S3J0B6_LINUN-AGRA3isoformX3-Lingula_unguis. doublet
A0A3B3XFS2-A0A3B3XFS2_9TELE-AGRA3-Poecilia_mexicana. doublet
A0A3B4XI57-A0A3B4XI57_SERLL-AGRA2-Seriola_lalandi_dorsalis. doublet
A0A3Q7XTM5-A0A3Q7XTM5_URSAR-AGRA3-Ursus_arctos_horribilis. doublet
A0A7K4KH41-A0A7K4KH41_9AVES-AGRD1protein-Crypturellus_soui. doublet
A0A7K4QK13-A0A7K4QK13_9TYRA-AGRD1protein-Neopipo_cinnamomea. doublet
A0A7K4T4B1-A0A7K4T4B1_9CHAR-AGRD1protein-Burhinus_bistriatus. doublet
A0A7K4ZI22-A0A7K4ZI22_9AVES-AGRD1protein-Centropus_unirufus. doublet
A0A7K5UZU9-A0A7K5UZU9_9CORV-AGRD1protein-Platysteira_castanea. doublet
A0A7K5Y2A7-A0A7K5Y2A7_9CHAR-AGRD1protein-Dromas_ardeola. doublet
A0A7K5Y7L5-A0A7K5Y7L5_9AVES-AGRD1protein-Pterocles_burchelli. doublet
A0A7K6JAN3-A0A7K6JAN3_9CORV-AGRD1protein-Machaerirhynchus_nigripectus. doublet
A0A7K6TSG3-A0A7K6TSG3_9AVES-AGRD1protein-Aegotheles_bennettii. doublet
A0A7K7KX31-A0A7K7KX31_9AVES-AGRD1pro

...

A0A6P9D9L7-A0A6P9D9L7_PANGU-AGRA1-Pantherophis_guttatus doublet
A0A7L3QSM4-A0A7L3QSM4_9SYLV-AGRA3protein-Cettia_cetti. doublet
A0A7L4L2V4-A0A7L4L2V4_9CORV-AGRA3protein-Callaeas_wilsoni doublet
A0A7M7LR07-A0A7M7LR07_NASVI-Uncharacterizedprotein-Nasonia_vitripennis doublet
K1Q644-K1Q644_CRAGI-Uncharacterizedprotein-Crassostrea_gigas doublet
A0A1S3NJI9-A0A1S3NJI9_SALSA-GR124-like-Salmo_salar doublet
A0A2K6Q648-A0A2K6Q648_RHIRO-Uncharacterizedprotein-Rhinopithecus_roxellana doublet
A0A2P4T5L1-A0A2P4T5L1_BAMTH-Uncharacterizedprotein-Bambusicola_thoracicus doublet
A0A3B3QFG4-A0A3B3QFG4_9TELE-AGRL1-like-Paramormyrops_kingsleyae. doublet
A0A3P8YTT0-A0A3P8YTT0_ESOLU-AGRF3-Esox_lucius doublet
A0A3Q3IIE6-A0A3Q3IIE6_MONAL-AGRA2-like-Monopterus_albus doublet
A0A6J0D343-A0A6J0D343_PERMB-AGRB2-Peromyscus_maniculatus_bairdii doublet
A0A6J0D694-A0A6J0D694_PERMB-GR126isoformX1-Peromyscus_maniculatus_bairdii doublet
A0A6J0D7H1-A0A6J0D7H1_PERMB-AGRG6-Peromyscus_maniculatus_bairdii doublet
A0A6J0D8F7-A0A6J

A0A6J0X823-A0A6J0X823_ODOVR-AGRL4isoformX2-Odocoileus_virginianus_texanus. doublet
A0A2U3WVY1-A0A2U3WVY1_ODORO-EGF,latrophilinandseventransmembranedomain-containingprotein1isoformX2-Odobenus_rosmarus_divergens doublet
A0A3B3QG91-A0A3B3QG91_9TELE-AGRL4-Paramormyrops_kingsleyae. doublet
A0A3P9L967-A0A3P9L967_ORYLA-AGRE5b,duplicate3-Oryzias_latipes doublet
A0A6I9ITZ6-A0A6I9ITZ6_CHRAS-EGF,latrophilinandseventransmembranedomain-containingprotein1-like-Chrysochloris_asiatica doublet
A0A6J0XAU1-A0A6J0XAU1_ODOVR-AGRL4isoformX1-Odocoileus_virginianus_texanus. doublet
A0A6P3IXN3-A0A6P3IXN3_BISBI-EGF,latrophilinandseventransmembranedomain-containingprotein1isoformX1-Bison_bison_bison. doublet
Q08DX2-Q08DX2_BOVIN-EGF,latrophilinandseventransmembranedomaincontaining1-Bos_taurus doublet
A0A2Y9J2V2-A0A2Y9J2V2_ENHLU-AGRL4isoformX2-Enhydra_lutris_kenyoni. doublet
H2PYJ4_A0A2J8IYQ9-H2PYJ4_PANTR-AGRB2-Pan_troglodytes doublet
H2QFK2-H2QFK2_PANTR-AGRL1-Pan_troglodytes doublet
H2QPJ0_A0A2J8PEX1-H2QPJ0_PANTR

A0A7K9KKV7-A0A7K9KKV7_9PASE-AGRL4protein-Dicaeum_eximium. doublet
A0A7L0P580-A0A7L0P580_9AVES-AGRL4protein-Mesembrinibis_cayennensis. doublet
A0A7L0UXY7-A0A7L0UXY7_9PASE-AGRL4protein-Leptocoma_aspasia. doublet
A0A7L2KPT0-A0A7L2KPT0_9PASS-AGRL4protein-Zosterops_hypoxanthus. doublet
A0A3B4ABI5-A0A3B4ABI5_9GOBI-Uncharacterizedprotein-Periophthalmus_magnuspinnatus. doublet
A0A7K4TN67-A0A7K4TN67_9CHAR-AGRL4protein-Burhinus_bistriatus. doublet
A0A7K6VT78-A0A7K6VT78_9PASS-AGRL4protein-Notiomystis_cincta. doublet
A0A7K7XFA8-A0A7K7XFA8_9PASS-AGRL4protein-Mohoua_ochrocephala. doublet
A0A7K4KVK1-A0A7K4KVK1_9AVES-AGRL4protein-Crypturellus_soui. doublet
A0A7K4LXI1-A0A7K4LXI1_9AVES-AGRL4protein-Crypturellus_undulatus. doublet
A0A7K6ZEV1-A0A7K6ZEV1_9AVES-AGRL4protein-Nothoprocta_pentlandii. doublet
A0A7K7AFP2-A0A7K7AFP2_9AVES-AGRL4protein-Nothoprocta_ornata. doublet
A0A7K7WA81-A0A7K7WA81_9AVES-AGRL4protein-Nothocercus_julius. doublet
A0A7K8QJJ0-A0A7K8QJJ0_9PASS-AGRL4protein-Smithornis_capensis. doubl

Sublist constructed for suffix = 'fragments' containing 524 files.
A0A803XZQ8-A0A803XZQ8_MELGA-CadherinEGFLAGseven-passG-typereceptor3-Meleagris_gallopavo doublet
A0A2Y9R5T3-A0A2Y9R5T3_TRIMA-LOWQUALITYPROTEIN:AGRE2-like-Trichechus_manatus_latirostris doublet
A0A3B4Y6Q3-A0A3B4Y6Q3_SERLL-AGRF3b-Seriola_lalandi_dorsalis. doublet
A0A224YYG0-A0A224YYG0_9ACAR-ProteincontainingCUB,7tm2andI-setdomain-Rhipicephalus_zambeziensis. doublet
A0A6J0ZFG0-A0A6J0ZFG0_ODOVR-AGRE3-Odocoileus_virginianus_texanus. doublet
A0A6J8ARM4-A0A6J8ARM4_MYTCO-AGRB2,Coadhesin,Thrombospondin-1,AGRB1,Hemicentin-1,AGRB3,Thrombospondin-2-Mytilus_coruscus doublet
A0A2I4D4H7-A0A2I4D4H7_9TELE-AGRG5-like-Austrofundulus_limnaeus. doublet
A0A8D9AJW0-A0A8D9AJW0_9HEMI-AGRA3-Cacopsylla_melanoneura. doublet
A0A6I9Y3R9-A0A6I9Y3R9_9SAUR-AGRD2-like-Thamnophis_sirtalis. doublet
V4B2B9-V4B2B9_LOTGI-Uncharacterizedprotein-Lottia_gigantea doublet
A0A369SID9-A0A369SID9_9METZ-AGRL3-Trichoplax_sp._H2. doublet
A0A671K9Q9-A0A671K9Q9_9TELE-AGRG

A0A6P7X5D4-A0A6P7X5D4_9AMPH-AGRG6-like-Microcaecilia_unicolor. doublet
A0A671QC34-A0A671QC34_9TELE-AGRG3-Sinocyclocheilus_anshuiensis. doublet
A0A139WAE3-A0A139WAE3_TRICA-GR124-likeProtein-Tribolium_castaneum doublet
A0A1A9URJ7-A0A1A9URJ7_GLOAU-Uncharacterizedprotein-Glossina_austeni doublet


In [ ]:
# Get the full profile for D1 : Q6QNK2.
